#   Introduction

# Context:
 It is important that credit card companies are able to recognize fraudulent credit card transaction so that customers are not charged for items that they did not purchase. This is a big problem in Sierra Leone.

# Content:
 I were hoping to real dataset from Sierra Leone but since it was not possible, I used dataset from kaggle in order to build a module so that in the further i can do the same thing for a real dataset of credit card frauds in Sierra Leone.

 The datasets contains transactions made by credit cards in September 2013 European cardholders. This dataset presents transactions that occurred in two days, where they have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced. 

 The positive class (frauds) account for 0.172% of all transactions. In this project, I will be predicting whether a transaction is fraud or not fraud by using predictive models.


 It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, i cannot provide the original features and more background information about the data. 

#The dataset given contains features V1 to V28 which are a result of PCA dimensionality reduction to protect user identities and sensitive features (according to the description) are the principal components obtained with PCA, the only features which have not been transformed with PCA are ‘Time’ and ‘Amount’. Feature ‘Time’ contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature ‘Amount’ is the transaction Amount this feature can be use for example-dependent cost-sensitive learning . Feature ‘Class’ is the response variable and it takes the value of 1 in the class if it is fraud and the value of 0 in the class if it is not fraud.

# Inspiration:
 Identify fraudulent credit card transaction.
 Given the class imbalance ratio, I  recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC).Confusion matrix accuracy is not meaningful for unbalance classification.    

# Acknowledgement:
 The dataset has been collected and analyzed during a research on big data mining and fraud detection.


In [ ]:
#Import the required libraries

import numpy as np
import pandas as pd
import sklearn
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42
LABELS = ["Normal", "Fraud"]
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot


In [ ]:
data = pd.read_csv('../input/creditcard_data.csv')
data.head()

In [ ]:
data.info()

# Exploratory Data Analysis

In [ ]:
data.isnull().values.any()

In [ ]:
data.describe()

In [ ]:
#Determine the number of fraud and valid transactions in the entire dataset

count_classes = pd.value_counts(data['Class'], sort = True)
count_classes.plot(kind = 'bar', rot=0)
plt.title("Transaction Class Distribution")
plt.xticks(range(2), LABELS)
plt.xlabel("Class")
plt.ylabel("Frequency");

In [ ]:
#Assigning the transaction class "0 = NORMAL  & 1 = FRAUD"
Normal = data[data['Class']==0]
Fraud = data[data['Class']==1]

In [ ]:
Normal.shape

In [ ]:
Fraud.shape

In [ ]:
#How different are the amount of money used in different transaction classes?

Normal.Amount.describe()

In [ ]:
#How different are the amount of money used in different transaction classes?

Fraud.Amount.describe()

In [ ]:
#Let's have a more graphical representation of the data

f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
f.suptitle('Amount per transaction by class')
bins = 50
ax1.hist(Fraud.Amount, bins = bins)
ax1.set_title('Fraud')
ax2.hist(Normal.Amount, bins = bins)
ax2.set_title('Normal')
plt.xlabel('Amount ($)')
plt.ylabel('Number of Transactions')
plt.xlim((0, 20000))
plt.yscale('log')
plt.show();

In [ ]:
#Graphical representation of the data

f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
f.suptitle('Time of transaction vs Amount by class')
ax1.scatter(Fraud.Time, Fraud.Amount)
ax1.set_title('Fraud')
ax2.scatter(Normal.Time, Normal.Amount)
ax2.set_title('Normal')
plt.xlabel('Time (in Seconds)')
plt.ylabel('Amount')
plt.show();

In [ ]:
init_notebook_mode(connected=True)
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
# Create a trace

trace = go.Scatter(
    x = Fraud.Time,
    y = Fraud.Amount,
    mode = 'markers'
)
data = [trace]

plotly.offline.iplot({
    "data": data
})

Doesn't seem like the time of transaction really matters here as per above observation. Now let us take a sample of the dataset for out modelling and prediction

In [ ]:
data1.shape

In [ ]:
#Determine the number of fraud and valid transactions in the dataset.

Fraud = data1[data1['Class']==1]
Valid = data1[data1['Class']==0]
outlier_fraction = len(Fraud)/float(len(Valid))

In [ ]:
#Now let us print the outlier fraction and no of Fraud and Valid Transaction cases

print(outlier_fraction)
print("Fraud Cases : {}".format(len(Fraud)))
print("Valid Cases : {}".format(len(Valid)))

In [ ]:
#Correlation Matrix

correlation_matrix = data1.corr()
fig = plt.figure(figsize=(12,9))
sns.heatmap(correlation_matrix,vmax=0.8,square = True)
plt.show()

The above correlation matrix shows that none of the V1 to V28 PCA components have any correlation to each other however if we observe Class has some form positive and negative correlations with the V components but has no correlation with Time and Amount.

In [ ]:
#Get all the columns from the dataframe

columns = data1.columns.tolist()
# Filter the columns to remove data we do not want 
columns = [c for c in columns if c not in ["Class"]]
# Store the variable we are predicting 
target = "Class"
# Define a random state 
state = np.random.RandomState(42)
X = data1[columns]
Y = data1[target]
X_outliers = state.uniform(low=0, high=1, size=(X.shape[0], X.shape[1]))
# Print the shapes of X & Y
print(X.shape)
print(Y.shape)

# Model Prediction:
Now it is time to start building the model .The types of algorithms that I am using to try to do anomaly detection on this dataset are as follows:

# Isolation Forest Algorithm: 
One of the new techniques to detect anomalies is called Isolation Forests. The algorithm is based on the fact that anomalies are data points that are few and different. As a result of these properties, anomalies are subjected to a mechanism called isolation. 

 This method is highly useful and is fundamentally different from all existing methods. It introduces the use of isolation as a more effective and efficient means to detect anomalies than the commonly used basic distance and density measures. Moreover, this method is an algorithm with a low linear time complexity and a small memory requirement. It builds a good performing model with a small number of trees using small sub-samples of fixed size, regardless of the size of a data set.

 Typical machine learning methods tend to work better when the patterns they try to learn are balanced, meaning the same amount of good and bad behaviors are present in the dataset.

 How Isolation Forests Work The Isolation Forest algorithm isolates observations by randomly selecting a feature and then randomly selecting a split value between the maximum and minimum values of the selected feature. The logic argument goes: isolating anomaly observations is easier because only a few conditions are needed to separate those cases from the normal observations. On the other hand, isolating normal observations require more conditions. Therefore, an anomaly score can be calculated as the number of conditions required to separate a given observation.

 The way that the algorithm constructs the separation is by first creating isolation trees, or random decision trees. Then, the score is calculated as the path length to isolate the observation.

# Local Outlier Factor(LOF) Algorithm:
 The LOF algorithm is an unsupervised outlier detection method which computes the local density deviation of a given data point with respect to its neighbors. It considers as outlier samples that have a substantially lower density than their neighbors. The number of neighbors considered, (parameter neighbors) is typically chosen 1) greater than the minimum number of objects a cluster has to contain, so that other objects can be local outliers relative to this cluster, and 2) smaller than the maximum number of close by objects that can potentially be local outliers. In practice, such information's are generally not available, and taking neighbors = 20 appears to work well in general.


Define the outlier detection methods

In [ ]:
#Define the outlier detection methods

classifiers = {
    "Isolation Forest":IsolationForest(n_estimators=100, max_samples=len(X), 
                                       contamination=outlier_fraction,random_state=state, verbose=0),
    "Local Outlier Factor":LocalOutlierFactor(n_neighbors=20, algorithm='auto', 
                                              leaf_size=30, metric='minkowski',
                                              p=2, metric_params=None, contamination=outlier_fraction),
    "Support Vector Machine":OneClassSVM(kernel='rbf', degree=3, gamma=0.1,nu=0.05, 
                                         max_iter=-1, random_state=state)
   
}

Fit the model

In [ ]:
#Fit the model

n_outliers = len(Fraud)
for i, (clf_name,clf) in enumerate(classifiers.items()):
    #Fit the data and tag outliers
    if clf_name == "Local Outlier Factor":
        y_pred = clf.fit_predict(X)
        scores_prediction = clf.negative_outlier_factor_
    elif clf_name == "Support Vector Machine":
        clf.fit(X)
        y_pred = clf.predict(X)
    else:    
        clf.fit(X)
        scores_prediction = clf.decision_function(X)
        y_pred = clf.predict(X)
    #Reshape the prediction values to 0 for Valid transactions , 1 for Fraud transactions
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1
    n_errors = (y_pred != Y).sum()
    # Run Classification Metrics
    print("{}: {}".format(clf_name,n_errors))
    print("Accuracy Score :")
    print(accuracy_score(Y,y_pred))
    print("Classification Report :")
    print(classification_report(Y,y_pred))

The neural network we fitted on the oversampled dataset has lower recall than the previous neural network. However, the precision has also significantly improved.


# Conclusion


When we are building a classifier, both precision and recall are important, but it all comes down to the task we are given. In this particular project, higher recall means we can detect more frauds which leads to a more safer platform, whereas higher precision means that we can correctly detect the fraud cases which leads to customer satisfaction (from avoiding their accounts getting blocked). Moreover, we can still improve on our oversampled dataset by removing outliers, and also fine tuning our neural network models.

I couldn’t use dataset of Sierra Leone, but in the further, since I have built a model, I am planning to use a real dataset from Cards Fraud in Sierra Leone.
